In [1]:
import numpy as np
import os
import pandas as pd
import copy 
import datetime 
data_path = '../../../code_data/arises/'
process_file = 'nips_23'

pid_list = [1000 + d for d in range(1, 13)]


In [2]:
pid2regular_df = {}
names = ['ts', 'glucose_level', 'isRealBGL', 'basal', 'meal', 
'bolus', 'correction_bolus', 'finger_stick',	'hypoStart', 
'EDA',	'SCL', 'SCR', 'HR', 'TEMP', 'ACC', 'RMSSD', 'SDNN', 'medianNNI', 'CVNNI', 'CVSD', 'pNNX', 
'meanHR', 'minHR', 'maxHR', 'VLF', 'LF', 'HF', 'LHR']
usecols = [i for i in range(28)]
for pid in pid_list:
    
    load_path = os.path.join(data_path, 'raw_data', f'dc_{pid}_data.csv')
    df = pd.read_csv(load_path, names=names, usecols=usecols, header=1)

    df['ts'] = pd.to_datetime(df['ts'], dayfirst=True, errors='coerce')
    # df = df.groupby('ts').sum()
    df = df.set_index('ts')
    df = df.sort_index()
    df['glucose_level'] = df.apply(lambda row: row['glucose_level'] if row['isRealBGL'] == True else np.nan,  axis = 1)
    df = df.drop(['hypoStart', 'isRealBGL'], axis=1)
    df = df.astype(np.float64)
    df = df.replace(0.0, np.nan)
    pid2regular_df[pid] = df
    print(f'{pid}, total_points:{len(df)}, start:{df.index[0]}, end:{df.index[-1]}')


1001, total_points:15831, start:2019-02-28 12:43:28, end:2019-04-24 11:51:50
1002, total_points:13215, start:2019-03-04 14:18:20, end:2019-04-19 11:25:42
1003, total_points:14896, start:2019-03-04 19:17:53, end:2019-04-25 12:31:25
1004, total_points:11517, start:2019-03-05 13:47:53, end:2019-04-14 13:25:57
1005, total_points:14064, start:2019-03-07 13:48:01, end:2019-04-25 09:41:10
1006, total_points:13464, start:2019-03-08 14:15:27, end:2019-04-24 08:08:52
1007, total_points:13066, start:2019-04-29 13:35:19, end:2019-06-13 22:18:07
1008, total_points:14307, start:2019-04-29 16:00:45, end:2019-06-18 08:08:44
1009, total_points:14201, start:2019-04-30 12:41:20, end:2019-06-18 19:59:44
1010, total_points:12911, start:2019-04-30 16:12:50, end:2019-06-14 12:01:44
1011, total_points:13018, start:2019-05-07 12:41:15, end:2019-06-21 17:25:02
1012, total_points:15797, start:2019-05-07 16:05:46, end:2019-07-01 12:23:44


In [3]:
# check
for pid in pid_list:
    delta_ts = pid2regular_df[pid].index[1:] - pid2regular_df[pid].index[:-1]
    print(pid, (delta_ts>datetime.timedelta(minutes=5, seconds=10)).sum())

1001 0
1002 0
1003 0
1004 0
1005 0
1006 0
1007 0
1008 0
1009 0
1010 0
1011 0
1012 0


In [4]:
# add time features

def add_time_attributes(pid2data):
    for pid in pid2data:
        data = pid2data[pid]

        temp = data.index.to_frame().loc[:, 'ts'].dt
        
        data['day_of_week'] = temp.dayofweek
        data['day_of_week'] = data['day_of_week'].astype(np.float64)

        data['hour'] = temp.hour
        data['hour'] = data['hour'].astype(np.float64)

        data['minute'] = temp.minute
        data['minute'] = data['minute'].astype(np.float64)

        
        data['timestamp'] = temp.hour * 3600 +\
                                temp.minute * 60 +\
                                temp.second
        data['timestamp'] = data['timestamp'].astype(np.float64)
        
        # new ————————————————————————
        seconds_in_day = 24*60*60

        data['sin_time'] = np.sin(2 * np.pi * data.timestamp / seconds_in_day)
        data['cos_time'] = np.cos(2 * np.pi * data.timestamp / seconds_in_day)
        data['sin_time'].astype(np.float64)
        data['cos_time'].astype(np.float64)
        # end ______________________
        
        
    return pid2data

In [5]:
pid2regular_df = add_time_attributes(pid2regular_df)


In [6]:
pid2regular_df[1001].sin_time.describe()

count    15831.000000
mean         0.000045
std          0.707334
min         -0.999977
25%         -0.712001
50%          0.007272
75%          0.712128
max          0.999976
Name: sin_time, dtype: float64

In [7]:
# add discrete y

def discrete(y):
    if y < 70:
        return 0
    elif 70<= y <=180:
        return 1
    elif y > 180:
        return 2
    else:
        return np.nan


def add_discrete_y(pid2data):
    for pid in pid2data:
        data = pid2data[pid]
        data['discrete_y'] = data['glucose_level']

        data['discrete_y'] = data['discrete_y'].apply(discrete)
        
    return pid2data

In [8]:
pid2regular_df = add_discrete_y(pid2regular_df)

In [9]:
pid2regular_df[1001].columns

Index(['glucose_level', 'basal', 'meal', 'bolus', 'correction_bolus',
       'finger_stick', 'EDA', 'SCL', 'SCR', 'HR', 'TEMP', 'ACC', 'RMSSD',
       'SDNN', 'medianNNI', 'CVNNI', 'CVSD', 'pNNX', 'meanHR', 'minHR',
       'maxHR', 'VLF', 'LF', 'HF', 'LHR', 'day_of_week', 'hour', 'minute',
       'timestamp', 'sin_time', 'cos_time', 'discrete_y'],
      dtype='object')

In [10]:
def add_indicate_for_nan(pid2data, ignore_indicate_set):
    for pid in pid2data:
        data = pid2data[pid]
        for attri in data:
            if attri in ignore_indicate_set:
                continue
            data[attri + '_indicate'] = 1
            data.loc[pd.notna(data[attri]), attri + '_indicate'] = 0
            if sum(data[attri + '_indicate']) != len(data[attri]) - sum(pd.notna(data[attri])):
                print('error')
    return pid2data

In [11]:
ignore_indicate_set = ['day_of_week', 'hour', 'minute',
       'timestamp', 'sin_time', 'cos_time',]
pid2regular_df = add_indicate_for_nan(pid2regular_df, ignore_indicate_set)

In [12]:
pid2train = {}
pid2valid = {}
pid2test = {}

for pid in pid_list:
    idxs = pid2regular_df[pid].index[pd.notna(pid2regular_df[pid]['glucose_level'])]

    idxs = idxs.to_frame()

    total_len = len(idxs)
    train_end = int(total_len * 0.6)
    valid_end = int(total_len * 0.8) 

    pid2train[pid] = pid2regular_df[pid].loc[:idxs.iloc[train_end][0]]

    pid2valid[pid] = pid2regular_df[pid].loc[idxs.iloc[train_end + 1][0]:idxs.iloc[valid_end][0]]

    pid2test[pid] = pid2regular_df[pid].loc[idxs.iloc[valid_end + 1][0]:]

    print(pid, total_len, len(pid2train[pid]), len(pid2valid[pid]), len(pid2test[pid]))

1001 15142 9636 3122 3073
1002 12882 7871 2635 2709
1003 14223 8952 2903 3041
1004 11165 6782 2354 2381
1005 13888 8437 2807 2820
1006 13062 8105 2692 2667
1007 11905 8202 2421 2443
1008 14200 8598 2870 2839
1009 13671 8614 2790 2797
1010 12736 7741 2587 2583
1011 12690 7805 2612 2601
1012 14334 9033 3650 3114


In [13]:
# calculate pearson
df = pd.concat(list(pid2train.values()))
df[['glucose_level', 'basal', 'meal', 'bolus', 'correction_bolus',
       'finger_stick', 'EDA', 'SCL', 'SCR', 'HR', 'TEMP', 'ACC', 'RMSSD', 'SDNN',
       'medianNNI', 'CVNNI', 'CVSD', 'pNNX', 'meanHR', 'minHR', 'maxHR', 'VLF',
       'LF', 'HF', 'LHR', 'timestamp']].corr(method ='pearson')['glucose_level'].sort_values()

SCL                -0.035496
medianNNI          -0.034121
SDNN               -0.025722
basal              -0.025605
VLF                -0.021262
LF                 -0.016292
LHR                -0.015930
EDA                -0.012553
timestamp          -0.011617
RMSSD              -0.011113
CVNNI              -0.010650
ACC                -0.009611
pNNX               -0.005775
HF                 -0.002610
CVSD               -0.000801
SCR                 0.003909
TEMP                0.006357
HR                  0.007584
maxHR               0.015800
meanHR              0.033335
minHR               0.033883
meal                0.134624
bolus               0.278311
correction_bolus    0.484299
finger_stick        0.934956
glucose_level       1.000000
Name: glucose_level, dtype: float64

In [14]:
pid2train[pid].columns

Index(['glucose_level', 'basal', 'meal', 'bolus', 'correction_bolus',
       'finger_stick', 'EDA', 'SCL', 'SCR', 'HR', 'TEMP', 'ACC', 'RMSSD',
       'SDNN', 'medianNNI', 'CVNNI', 'CVSD', 'pNNX', 'meanHR', 'minHR',
       'maxHR', 'VLF', 'LF', 'HF', 'LHR', 'day_of_week', 'hour', 'minute',
       'timestamp', 'sin_time', 'cos_time', 'discrete_y',
       'glucose_level_indicate', 'basal_indicate', 'meal_indicate',
       'bolus_indicate', 'correction_bolus_indicate', 'finger_stick_indicate',
       'EDA_indicate', 'SCL_indicate', 'SCR_indicate', 'HR_indicate',
       'TEMP_indicate', 'ACC_indicate', 'RMSSD_indicate', 'SDNN_indicate',
       'medianNNI_indicate', 'CVNNI_indicate', 'CVSD_indicate',
       'pNNX_indicate', 'meanHR_indicate', 'minHR_indicate', 'maxHR_indicate',
       'VLF_indicate', 'LF_indicate', 'HF_indicate', 'LHR_indicate',
       'discrete_y_indicate'],
      dtype='object')

In [15]:
pid_attri2mean_std = {}
attris_should_norm = ['glucose_level', 'basal', 'meal', 'bolus', 'correction_bolus',
       'finger_stick', 'EDA', 'SCL', 'SCR', 'HR', 'TEMP', 'ACC', 'RMSSD', 'SDNN',
       'medianNNI', 'CVNNI', 'CVSD', 'pNNX', 'meanHR', 'minHR', 'maxHR', 'VLF',
       'LF', 'HF', 'LHR', 'timestamp']
for pid in pid_list:
    for attri in attris_should_norm:
        mean = pid2train[pid][attri].mean()
        std = pid2train[pid][attri].std()
        if std is np.NaN or std is pd.NaT or std == 0:
            std = 1e-6
        pid_attri2mean_std[(pid, attri)] = (mean, std)

if not os.path.exists(os.path.join(data_path, process_file)):
    os.makedirs(os.path.join(data_path, process_file))
np.save(os.path.join(data_path, process_file, 'pid_attri2mean_std.npy'), pid_attri2mean_std)

In [16]:
def norm_data(pid2data, pid_attri2mean_std, attris_should_norm):

    new_pid2data = {}
    for pid in pid2data:

        new_pid2data[pid] = copy.deepcopy(pid2data[pid]) 

        for attri in attris_should_norm:

            mean, std = pid_attri2mean_std[(pid, attri)]

            if std <= 1e-6:
                new_pid2data[pid][attri] = new_pid2data[pid][attri] / mean
            else:
                new_pid2data[pid][attri] = (new_pid2data[pid][attri] - mean) / std          

    return new_pid2data

In [17]:
pid2train_norm = norm_data(pid2train, pid_attri2mean_std, attris_should_norm)
pid2valid_norm = norm_data(pid2valid, pid_attri2mean_std, attris_should_norm)
pid2test_norm = norm_data(pid2test, pid_attri2mean_std, attris_should_norm)

In [18]:
attri_list_wo_glucose = [
    'basal', 'meal', 'bolus', 'correction_bolus',
    'finger_stick', 'EDA', 'SCL', 'SCR', 'HR', 'TEMP', 'ACC', 'RMSSD', 'SDNN',
    'medianNNI', 'CVNNI', 'CVSD', 'pNNX', 'meanHR', 'minHR', 'maxHR', 'VLF',
    'LF', 'HF', 'LHR', 'day_of_week', 'hour', 'minute', 'timestamp',
    'sin_time', 'cos_time', 'glucose_level_indicate',
    'basal_indicate', 'meal_indicate', 'bolus_indicate',
    'correction_bolus_indicate', 'finger_stick_indicate', 'EDA_indicate',
    'SCL_indicate', 'SCR_indicate', 'HR_indicate', 'TEMP_indicate',
    'ACC_indicate', 'RMSSD_indicate', 'SDNN_indicate', 'medianNNI_indicate',
    'CVNNI_indicate', 'CVSD_indicate', 'pNNX_indicate', 'meanHR_indicate',
    'minHR_indicate', 'maxHR_indicate', 'VLF_indicate', 'LF_indicate',
    'HF_indicate', 'LHR_indicate' 
]

temp = []

for idx, attri in enumerate(attri_list_wo_glucose):
    temp.append({'attri':attri, 'idx':idx}, )
attri2idx = pd.DataFrame(temp).set_index('attri')
idx2attri = pd.DataFrame(temp).set_index('idx')
print(attri2idx)
print(idx2attri)

save_path = os.path.join(data_path, process_file)

attri2idx.to_pickle(os.path.join(save_path,'attri2idx.pkl'))
idx2attri.to_pickle(os.path.join(save_path,'idx2attri.pkl'))


                           idx
attri                         
basal                        0
meal                         1
bolus                        2
correction_bolus             3
finger_stick                 4
EDA                          5
SCL                          6
SCR                          7
HR                           8
TEMP                         9
ACC                         10
RMSSD                       11
SDNN                        12
medianNNI                   13
CVNNI                       14
CVSD                        15
pNNX                        16
meanHR                      17
minHR                       18
maxHR                       19
VLF                         20
LF                          21
HF                          22
LHR                         23
day_of_week                 24
hour                        25
minute                      26
timestamp                   27
sin_time                    28
cos_time                    29
glucose_

In [19]:
for pid in pid_list:
    pid2train_norm[pid] = pid2train_norm[pid].fillna(0.0)
    pid2valid_norm[pid] = pid2valid_norm[pid].fillna(0.0)
    pid2test_norm[pid] = pid2test_norm[pid].fillna(0.0)

In [20]:
# generate training data


def gen_final(pid2data, attri2idx,  n_prev, pred_window, name, valid_points = 24): 
    pid2data_npy = {}
    for pid in pid_list:
        data = pid2data[pid]
        
        y_list = []
        y_dis_list = []
        target_cos_time_list = []
        target_sin_time_list = []
        target_total_seconds_list = []
        attri_X_list = []
        attri_X_tar_list = []
        y_tar_list = []

        glucose_level_X_list = []
        
        seq_st_ed_list = []
        
        seq_st, seq_ed = None, None


        for idx in range(len(data) - n_prev - pred_window + 1):
            glucose_level_y = data.iloc[idx + n_prev + pred_window - 1]['glucose_level']
            glucose_level_y_indicate = data.iloc[idx + n_prev + pred_window - 1]['glucose_level_indicate']
            if abs(glucose_level_y_indicate - 1) < 1e-6:
                continue

            # Resonable
            num_valids = int(valid_points - sum(data.iloc[idx + n_prev - valid_points : idx + n_prev]['glucose_level_indicate']))
            if num_valids == 0:
                if seq_st is not None and seq_ed - seq_st >= 2:
                    seq_st_ed_list.append([seq_st, seq_ed])
                seq_st, seq_ed = None, None
                continue
            # end

            
            y_list.append(glucose_level_y)
            y_dis_list.append(data.iloc[idx + n_prev + pred_window - 1]['discrete_y'])
            target_cos_time_list.append(data.iloc[idx + n_prev + pred_window - 1]['cos_time'])
            target_sin_time_list.append(data.iloc[idx + n_prev + pred_window - 1]['sin_time'])
            target_total_seconds_list.append(data.iloc[idx + n_prev + pred_window - 1]['timestamp'])
            glucose_level_X = np.array(data.iloc[idx: idx + n_prev]['glucose_level'].to_numpy(), dtype=np.float32)


            attri_X = np.array(data.iloc[idx: idx + n_prev][list(attri2idx.index)].to_numpy(), dtype=np.float32)          

            glucose_level_X_list.append(np.expand_dims(glucose_level_X, axis=0))

            attri_X_list.append(np.expand_dims(attri_X, axis=0))
            
            
            # new
            attri_X_tar = np.array(data.iloc[idx + n_prev: idx + n_prev + pred_window][list(attri2idx.index)].to_numpy(), dtype=np.float32)          
            attri_X_tar_list.append(np.expand_dims(attri_X_tar, axis=0))
            y_tar = data.iloc[idx + n_prev: idx + n_prev + pred_window]['glucose_level']
            y_tar_list.append(np.expand_dims(y_tar, axis=0)) 


            if seq_st is None:
                seq_st = len(y_list) - 1
            else:
                seq_ed = len(y_list) - 1

        if seq_st is not None and seq_ed - seq_st >= 2:
            seq_st_ed_list.append([seq_st, seq_ed])

        pid2data_npy[pid] = {}
        pid2data_npy[pid]['glucose_level_X'] = np.concatenate(glucose_level_X_list, axis=0)

        pid2data_npy[pid]['attri_X'] = np.concatenate(attri_X_list, axis=0)
        pid2data_npy[pid]['y'] = np.array(y_list, dtype=np.float32)
        pid2data_npy[pid]['y_dis'] = np.array(y_dis_list, dtype=np.int32)
        pid2data_npy[pid]['target_cos_time'] = np.array(target_cos_time_list, dtype=np.float32)
        pid2data_npy[pid]['target_sin_time'] = np.array(target_sin_time_list, dtype=np.float32)
        pid2data_npy[pid]['target_timestamp'] = np.array(target_total_seconds_list, dtype=np.float32)
        pid2data_npy[pid]['mean'] = pid_attri2mean_std[(pid, 'glucose_level')][0]
        pid2data_npy[pid]['std'] = pid_attri2mean_std[(pid, 'glucose_level')][1]

        pid2data_npy[pid]['seq_st_ed_list'] = np.array(seq_st_ed_list, dtype=np.int32)
        
        pid2data_npy[pid]['attri_X_tar'] = np.concatenate(attri_X_tar_list, axis=0) 
        pid2data_npy[pid]['y_tar'] = np.concatenate(y_tar_list, axis=0)

        print(pid, pid2data_npy[pid]['glucose_level_X'].shape, pid2data_npy[pid]['attri_X'].shape)
        print('    ',len(y_list), seq_st_ed_list)
    save_path = os.path.join(data_path, process_file)
    np.save(os.path.join(save_path, f'{name}_{n_prev}_{pred_window}.npy'), pid2data_npy)




In [21]:
gen_final(pid2train_norm, attri2idx,  n_prev=48, pred_window=6, name='train_pid2data_npy', valid_points = 24)

1001 (9006, 48) (9006, 48, 55)
     9006 [[0, 926], [927, 2665], [2666, 4786], [4787, 5197], [5198, 5469], [5470, 7772], [7773, 9005]]
1002 (7679, 48) (7679, 48, 55)
     7679 [[0, 1644], [1645, 3969], [3970, 6820], [6821, 7678]]
1003 (8445, 48) (8445, 48, 55)
     8445 [[0, 543], [544, 2590], [2591, 2743], [2744, 3403], [3404, 5252], [5253, 5455], [5456, 7654], [7655, 8214], [8215, 8444]]
1004 (6639, 48) (6639, 48, 55)
     6639 [[0, 2805], [2806, 5640], [5641, 6638]]
1005 (8270, 48) (8270, 48, 55)
     8270 [[0, 2511], [2512, 5216], [5217, 7865], [7866, 8269]]
1006 (7769, 48) (7769, 48, 55)
     7769 [[0, 1143], [1144, 2589], [2590, 3333], [3334, 5270], [5271, 7768]]
1007 (7032, 48) (7032, 48, 55)
     7032 [[0, 54], [55, 1660], [1661, 2541], [2542, 2757], [2758, 3243], [3244, 3742], [3743, 4144], [4145, 4423], [4424, 5179], [5180, 6739], [6740, 7031]]
1008 (8460, 48) (8460, 48, 55)
     8460 [[0, 2804], [2805, 5600], [5601, 8438], [8439, 8459]]
1009 (8135, 48) (8135, 48, 55)
     81

In [22]:
gen_final(pid2train_norm, attri2idx,  n_prev=48, pred_window=12, name='train_pid2data_npy', valid_points = 24)

1001 (8981, 48) (8981, 48, 55)
     8981 [[0, 920], [921, 2659], [2660, 4774], [4775, 5184], [5185, 5456], [5457, 7753], [7754, 8980]]
1002 (7669, 48) (7669, 48, 55)
     7669 [[0, 1644], [1645, 3963], [3964, 6816], [6817, 7668]]
1003 (8417, 48) (8417, 48, 55)
     8417 [[0, 541], [542, 2588], [2589, 2741], [2742, 3395], [3396, 5238], [5239, 5435], [5436, 7638], [7639, 8192], [8193, 8416]]
1004 (6627, 48) (6627, 48, 55)
     6627 [[0, 2805], [2806, 5634], [5635, 6626]]
1005 (8264, 48) (8264, 48, 55)
     8264 [[0, 2511], [2512, 5216], [5217, 7865], [7866, 8263]]
1006 (7751, 48) (7751, 48, 55)
     7751 [[0, 1137], [1138, 2583], [2584, 3321], [3322, 5258], [5259, 7750]]
1007 (6972, 48) (6972, 48, 55)
     6972 [[0, 48], [49, 1648], [1649, 2523], [2524, 2733], [2734, 3213], [3214, 3706], [3707, 4102], [4103, 4375], [4376, 5125], [5126, 6685], [6686, 6971]]
1008 (8454, 48) (8454, 48, 55)
     8454 [[0, 2804], [2805, 5600], [5601, 8438], [8439, 8453]]
1009 (8117, 48) (8117, 48, 55)
     81

In [23]:
gen_final(pid2valid_norm, attri2idx,  n_prev=48, pred_window=6, name='valid_pid2data_npy', valid_points = 24)

1001 (2988, 48) (2988, 48, 55)
     2988 [[0, 1492], [1493, 1960], [1961, 2987]]
1002 (2517, 48) (2517, 48, 55)
     2517 [[0, 1835], [1836, 2516]]
1003 (2786, 48) (2786, 48, 55)
     2786 [[0, 1459], [1460, 2785]]
1004 (2169, 48) (2169, 48, 55)
     2169 [[0, 1325], [1326, 1700], [1701, 2168]]
1005 (2724, 48) (2724, 48, 55)
     2724 [[0, 2270], [2271, 2723]]
1006 (2558, 48) (2558, 48, 55)
     2558 [[0, 242], [243, 2557]]
1007 (2328, 48) (2328, 48, 55)
     2328 [[0, 2327]]
1008 (2786, 48) (2786, 48, 55)
     2786 [[0, 2775], [2776, 2785]]
1009 (2681, 48) (2681, 48, 55)
     2681 [[0, 2680]]
1010 (2492, 48) (2492, 48, 55)
     2492 [[0, 814], [815, 2491]]
1011 (2480, 48) (2480, 48, 55)
     2480 [[0, 851], [852, 2479]]
1012 (2808, 48) (2808, 48, 55)
     2808 [[0, 2610], [2611, 2807]]


In [24]:
gen_final(pid2valid_norm, attri2idx,  n_prev=48, pred_window=12, name='valid_pid2data_npy', valid_points = 24)

1001 (2976, 48) (2976, 48, 55)
     2976 [[0, 1492], [1493, 1954], [1955, 2975]]
1002 (2505, 48) (2505, 48, 55)
     2505 [[0, 1829], [1830, 2504]]
1003 (2774, 48) (2774, 48, 55)
     2774 [[0, 1453], [1454, 2773]]
1004 (2157, 48) (2157, 48, 55)
     2157 [[0, 1319], [1320, 1694], [1695, 2156]]
1005 (2718, 48) (2718, 48, 55)
     2718 [[0, 2270], [2271, 2717]]
1006 (2553, 48) (2553, 48, 55)
     2553 [[0, 2552]]
1007 (2322, 48) (2322, 48, 55)
     2322 [[0, 2321]]
1008 (2780, 48) (2780, 48, 55)
     2780 [[0, 2775], [2776, 2779]]
1009 (2675, 48) (2675, 48, 55)
     2675 [[0, 2674]]
1010 (2488, 48) (2488, 48, 55)
     2488 [[0, 2487]]
1011 (2469, 48) (2469, 48, 55)
     2469 [[0, 846], [847, 2468]]
1012 (2796, 48) (2796, 48, 55)
     2796 [[0, 2604], [2605, 2795]]


In [25]:
gen_final(pid2test_norm, attri2idx,  n_prev=48, pred_window=6, name='test_pid2data_npy', valid_points = 24)

1001 (2974, 48) (2974, 48, 55)
     2974 [[0, 1130], [1131, 2973]]
1002 (2517, 48) (2517, 48, 55)
     2517 [[0, 385], [386, 2516]]
1003 (2778, 48) (2778, 48, 55)
     2778 [[0, 403], [404, 1428], [1429, 1936], [1937, 2777]]
1004 (2167, 48) (2167, 48, 55)
     2167 [[0, 1860], [1861, 2059], [2060, 2166]]
1005 (2722, 48) (2722, 48, 55)
     2722 [[0, 2317], [2318, 2721]]
1006 (2553, 48) (2553, 48, 55)
     2553 [[0, 432], [433, 2552]]
1007 (2326, 48) (2326, 48, 55)
     2326 [[0, 63], [64, 2325]]
1008 (2786, 48) (2786, 48, 55)
     2786 [[0, 2785]]
1009 (2680, 48) (2680, 48, 55)
     2680 [[0, 2679]]
1010 (2493, 48) (2493, 48, 55)
     2493 [[0, 877], [878, 2492]]
1011 (2478, 48) (2478, 48, 55)
     2478 [[0, 1164], [1165, 2477]]
1012 (2801, 48) (2801, 48, 55)
     2801 [[0, 2372], [2373, 2549], [2550, 2800]]


In [26]:
gen_final(pid2test_norm, attri2idx,  n_prev=48, pred_window=12, name='test_pid2data_npy', valid_points = 24)

1001 (2968, 48) (2968, 48, 55)
     2968 [[0, 1130], [1131, 2967]]
1002 (2505, 48) (2505, 48, 55)
     2505 [[0, 379], [380, 2504]]
1003 (2760, 48) (2760, 48, 55)
     2760 [[0, 403], [404, 1422], [1423, 1924], [1925, 2759]]
1004 (2149, 48) (2149, 48, 55)
     2149 [[0, 1854], [1855, 2047], [2048, 2148]]
1005 (2716, 48) (2716, 48, 55)
     2716 [[0, 2317], [2318, 2715]]
1006 (2541, 48) (2541, 48, 55)
     2541 [[0, 426], [427, 2540]]
1007 (2320, 48) (2320, 48, 55)
     2320 [[0, 63], [64, 2319]]
1008 (2780, 48) (2780, 48, 55)
     2780 [[0, 2779]]
1009 (2674, 48) (2674, 48, 55)
     2674 [[0, 2673]]
1010 (2487, 48) (2487, 48, 55)
     2487 [[0, 877], [878, 2486]]
1011 (2468, 48) (2468, 48, 55)
     2468 [[0, 1160], [1161, 2467]]
1012 (2783, 48) (2783, 48, 55)
     2783 [[0, 2366], [2367, 2537], [2538, 2782]]
